In [1]:
pip install keras_nlp

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import numpy as np
import os

In [3]:
texts = ""

for(dirpath, dirnames, filenames) in os.walk("/poems"):
  for filename in filenames:
    if filename.endswith('.txt'):
        # print(filename)
        path = os.path.join(dirpath, filename)
        with open(path,'r') as file_stream:
            texts += file_stream.read()
    
print(texts[0:1000])


    Dans son cadre terni, le pâle Christ d'ivoire,
    Cloué les bras en croix sur son étoffe noire,
                Redoublait de pâleur;
    Et comme au Golgotha, dans sa dure agonie,
    Les muscles en relief de sa face jaunie
                Se tordaient de douleur.Les Éditions de _Plaute_ et de _Christophe-Colomb_, comédies en 3 actes
et en vers, et de _Baudouin_, _empereur_, tragédie en 3 actes, sont à
refaire, ayant été détruites dans un incendie.    Partout est engagé le combat redoutable;
    A l'heure harmonieuse où la terre s'endort,
    Il rend la nuit sinistre et l'ombre épouvantable,
    Tout brin d'herbe est un champ de carnage et de mort.Mon esprit, comme mes vertèbres,
Invoque ardemment le repos;
Le cœur plein de songes funèbres,    Toujours vous trouveriez, sous cette architecture,
    Au milieu de la fange et de la pourriture,
    Dans le suaire usé le cadavre tout droit,    Ces gens qui font des tours en plein air
    Commencent à être rares à Paris
    Dans ma jeun

In [4]:
text_list = texts.split('.')
text_list = list(filter(None, text_list))

import random
random.shuffle(text_list)
length = len(text_list)
#split the dataset 
text_train = text_list[:int(0.7*length)]
text_test = text_list[int(0.7*length):int(0.85*length)]
text_valid = text_list[int(0.85*length):]

In [5]:
from tensorflow.keras.layers import TextVectorization

#def custom_standardization(input_string):
#    sentence = tf.strings.lower(input_string)
#    sentence = tf.strings.regex_replace(sentence, "\n", " ")
#    return sentence

maxlen = 50
# You can also set calculate the longest sentence in the data - 25 in this case
# maxlen = len(max(text_list).split(' ')) 

vectorize_layer = TextVectorization(
#    standardize = custom_standardization,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(text_list)
vocab = vectorize_layer.get_vocabulary()
vocab_size = len(vocab)
index_lookup = dict(zip(range(len(vocab)), vocab))   
#example vectorrizing words
print(vectorize_layer(['hello world!']))


tf.Tensor(
[[   1 8730    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0]], shape=(1, 51), dtype=int64)


In [6]:
#DataSet creation
def create_dataset(text, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(text)
  dataset = dataset.shuffle(buffer_size=256)
  dataset = dataset.batch(batch_size)
  return dataset 

def preprocess_text(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

batch_size = 64

train_dataset = create_dataset(text_train, batch_size)

valid_dataset = create_dataset(text_valid, batch_size)

test_dataset = create_dataset(text_test, batch_size)

#preprocess text
train_dataset = train_dataset.map(preprocess_text)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_text)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

valid_dataset = valid_dataset.map(preprocess_text)
valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)


In [7]:
embed_dim = 128
num_heads = 4

def create_model():
    inputs = keras.layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(vocab_size, maxlen, embed_dim)(inputs)
    decoder = keras_nlp.layers.TransformerDecoder(intermediate_dim=embed_dim, 
                                                            num_heads=num_heads, 
                                                            dropout=0.5)(embedding_layer)
    
    outputs = keras.layers.Dense(vocab_size, activation='softmax')(decoder)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer="adam", 
        loss='sparse_categorical_crossentropy',
        metrics=[keras_nlp.metrics.Perplexity(), 'accuracy']
    )
    return model

model = create_model()
model.summary()

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 token_and_position_embeddin  (None, 50, 128)          5431296   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, 50, 128)          99584     
 ormerDecoder)                                                   
                                                                 
 dense (Dense)               (None, 50, 42382)         5467278   
                                                                 
Total params: 10,998,158
Trainable params: 10,998,158
Non-trainable params: 0
_________________________________________________

In [8]:
#Custom Callback
class TextSampler(keras.callbacks.Callback):
    def __init__(self, start_prompt, max_tokens):
        self.start_prompt = start_prompt
        self.max_tokens = max_tokens
        
    # Helper method to choose a word from the top K probable words with respect to their probabilities
    # in a sequence
    def sample_token(self, logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

    def on_epoch_end(self, epoch, logs=None):
        decoded_sample = self.start_prompt
        
        for i in range(self.max_tokens-1):
            tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
            predictions = self.model.predict([tokenized_prompt], verbose=0)
            # To find the index of the next word in the prediction array.
            # The tokenized prompt is already shorter than the original decoded sample
            # by one, len(decoded_sample.split()) is two words ahead - so we remove 1 to get
            # the next word in the sequence
            sample_index = len(decoded_sample.strip().split())-1
            
            sampled_token = self.sample_token(predictions[0][sample_index])
            sampled_token = index_lookup[sampled_token]
            decoded_sample += " " + sampled_token
            
        print(f"\nSample text:\n{decoded_sample}...\n")

# First 5 words of a random sentence to be used as a seed
random_sentence = ' '.join(random.choice(text_valid).replace('\n', ' ').split(' ')[:4])
sampler = TextSampler(random_sentence, 30)
reducelr = keras.callbacks.ReduceLROnPlateau(patience=10, monitor='val_loss')

In [9]:
model = create_model()
history = model.fit(train_dataset, 
                    validation_data=valid_dataset,
                    epochs=3, 
                    callbacks=[sampler, reducelr])

Epoch 1/3
155/155 [==============================] - ETA: 0s - loss: 5.2729 - perplexity: 194.9877 - accuracy: 0.4785
Sample text:
  Beauté à de et de et et le les les le et les le le le le et et les de le le et le la de la et et et...

155/155 [==============================] - 239s 2s/step - loss: 5.2729 - perplexity: 194.9877 - accuracy: 0.4785 - val_loss: 3.8981 - val_perplexity: 49.3091 - val_accuracy: 0.5007 - lr: 0.0010
Epoch 2/3
155/155 [==============================] - ETA: 0s - loss: 3.6970 - perplexity: 40.3267 - accuracy: 0.5078
Sample text:
  Beauté à la mer à sa main des pieds  les yeux en son front dans un ciel  de la mer  de sa voix  qui ne te fait...

155/155 [==============================] - 243s 2s/step - loss: 3.6970 - perplexity: 40.3267 - accuracy: 0.5078 - val_loss: 3.6764 - val_perplexity: 39.5046 - val_accuracy: 0.5183 - lr: 0.0010
Epoch 3/3
155/155 [==============================] - ETA: 0s - loss: 3.3251 - perplexity: 27.8030 - accuracy: 0.5265
Sample text:

In [13]:
def sample_token(logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

def generate_text(prompt, response_length=20):
    decoded_sample = prompt
    for i in range(response_length-1):
        tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
        predictions = model.predict([tokenized_prompt], verbose=0)
        sample_index = len(decoded_sample.strip().split())-1

        sampled_token = sample_token(predictions[0][sample_index])
        sampled_token = index_lookup[sampled_token]
        decoded_sample += " " + sampled_token
    return decoded_sample

In [37]:
print(generate_text("J'aime les fleurs", 15))
print(generate_text("Combien d'entre vous", 15))
print(generate_text("Le corbeau sur un arbre perché tenait en son bec un", 4))
print(generate_text("Ce matin là, la première chose", 10))

J'aime les fleurs sans le ciel et la terre en vain le soleil dans les jours du
Combien d'entre vous vous en un soir le temps que nous avons la vie à la vie
Le corbeau sur un arbre perché tenait en son bec un jour à son
Ce matin là, la première chose de sa tête dans le vent et la vie
